In [1]:
import sys
import os
import pickle as pkl
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

sys.path.append("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/Splycer")
from record_set import RecordDataFrame
from pairs_set import PairsCSR
from feature_engineer import FeatureEngineer
from xgboost_match import XGBoostMatch
os.chdir("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/example")

In [2]:
#Load in the census data.
delaware_1910 = RecordDataFrame(2, pd.read_csv("delaware_1910.csv", index_col="index"))
delaware_1920 = RecordDataFrame(3, pd.read_csv("delaware_1920.csv", index_col="index"))

In [3]:
#Load in the compares set
compares = pd.read_csv("delaware_compares.csv")
compares = PairsCSR(1,2, compares.index_1910.values, compares.index_1920.values, np.ones(compares.shape[0]))

In [4]:
#Construct the feature engineer
fe = FeatureEngineer()
#FIXME add parent geo-coordinates
cols = ["marstat", "race", "rel", "mbp", "fbp", "first_sdxn", "last_sdxn", "bp", "county",
        "immigration", "birth_year", ["res_lat", "res_lon"], ["bp_lat", "bp_lon"],
        [f"first_vec{i}" for i in range(2, 202)], [f"last_vec{i}" for i in range(2,202)],
        "first", "last",
        "first", "last"
       ]

col_comps = ["exact match"] * 9
col_comps.extend(["euclidean dist"] * 6)
#col_comps.extend(["geo dist"] * 2)
col_comps.extend(["jw"] * 2)
col_comps.extend(["trigram"] * 2)

col_args = list({} for i in range(5))
col_args.extend([{"comm_weight": "d", "comm_col": "first_comm"}, {"comm_weight": "d", "comm_col": "last_comm"},
                 {"comm_weight": "d", "comm_col": "bp_comm"}])
col_args.extend(list({} for i in range(7)))
col_args.extend([{"comm_weight": "d", "comm_col": "first_comm"}, {"comm_weight": "d", "comm_col": "last_comm"}] * 2)
assert len(cols) == len(col_comps) == len(col_args)

In [5]:
for i,j,k in zip(cols, col_comps, col_args):
    fe.add_comparison(i, j, k)

In [6]:
#Construct XGBoostMatch
with open("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/model.xgboost", "rb") as file:
    model = pkl.load(file)
model.get_booster().feature_names = [f"f{i}" for i in range(19)]

In [7]:
xgb = XGBoostMatch(delaware_1910, delaware_1920, compares, fe, model)
xgb.run("test.csv")

R:\JoePriceResearch\Python\AnacondaNew\envs\ml_rec_env\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Numpy has detected that you may be viewing or writing to an array returned by selecting multiple fields in a structured array. 

This code may break in numpy 1.16 because this will return a view instead of a copy -- see release notes for details.
  
R:\JoePriceResearch\Python\AnacondaNew\envs\ml_rec_env\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in subtract
  


TypeError: str argument expected

In [ ]:
delaware_1910.df.head()

In [ ]:
delaware_1910.get_record(8361303)